In [ ]:
!pip install natasha slovnet navec -q

# Готовые инструменты для решения различных задач

До этого момента обычно мы пользовались разными библиотеками для решения наших задач: сегментация предложения и деление на токены - nltk, лемматизация и работа с морфологией - mystem или pymorphy, работа с синтаксисом - udpipe и тд. Но работа с разными библиотеками может вызывать трудности: различия в подходах, в формате ввода и вывода данных. Может сделать одну библиотеку для решения всех задач? Так и была придумана она - natasha.

Что она умеет:
- сегментация текста на предложения/токены
- морфологический анализ
- синтаксический анализ
- поиск именнованных сущностей
- создание собственных грамматик и правил
- использование предобученных эмбеддингов

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

import nltk
nltk.download('punkt')

Мы можем выбрать те элементы, которые нужны именно нам для решения задачи. Для начала их нужно инициализировать: сегментатор для деления на токены, морфологический и синтаксический парсеры, тэггер для NER.

Все теггеры основаны на предобученных эмбеддингах из класса NewsEmbedding - это обученный на русских новостях Glove.

## Natasha как таковая

In [ ]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

morph_vocab = MorphVocab()
names_extractor = NamesExtractor(morph_vocab)

Объект класса Doc - это что-то типа скрытого пайплайна: мы ему говорим, какие этапы ему нужно пройти.

In [ ]:
text = 'У моего любимого певца Егора Крида вышел новый трек. Буду его слушать весь день!'
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)

Некоторые этапы не работают без предыдущих.

In [ ]:
text = 'У моего любимого певца Егора Крида вышел новый трек. Буду его слушать весь день!'
doc = Doc(text)

try:
    morph = doc.tag_morph(morph_tagger)
except Exception as err:
    print('Проблема:', err)

#### Сегментация

In [ ]:
print(doc.tokens[:5])
print(doc.sents[:5])

In [ ]:
for i, s in enumerate(doc.sents):
    print("\n-- Sentence %d --" % i)
    for t in s.tokens:
        print(t.text, t.pos, sep="\t")

#### Лемматизация

Можно лемматизировать просто тексты:

In [ ]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)

{_.text: _.lemma for _ in doc.tokens}

А можно лемматизировать сущности, которым часто требуется другая обработка:

In [ ]:
doc.tag_ner(ner_tagger)
for span in doc.spans:
    span.normalize(morph_vocab)

{_.text: _.normal for _ in doc.spans}

__Задание 1:__ Как думаете, какие ещё сущности natasha умеет находить? Придумайте свои примеры, в которых находятся эти сущности.

In [ ]:
### Задание 1 ###
### Your code here ###

__Задание 2:__ Придумайте такой пример, в котором есть какая-то сущность, которая неверно лемматизируется с помощью token.lemmatize, но в span.normalize верно

In [ ]:
### Задание 2 ###
### Your code here ###

#### Морфология

In [ ]:
text = 'У моего любимого певца Егора Крида вышел новый трек. Буду его слушать весь день!'
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)

In [ ]:
sent = doc.sents[0]
sent.morph.print()

In [ ]:
sent.morph.tokens[:2]

#### Синтаксис

In [ ]:
sent.syntax.print()

In [ ]:
sent.syntax.tokens[:2]

#### NER

In [ ]:
ner_tagger = NewsNERTagger(emb)
doc.tag_ner(ner_tagger)
doc.ner.print()

In [ ]:
doc.ner

Но на самом деле Natasha - это обертка для более маленьких библиотек, каждая из которых отвечает за что-то своё.

## Razdel
По сути это библиотека-токенизатор. Она построена на правилах, основнанных на стандартах русской письменной речи.

In [ ]:
from razdel import tokenize, sentenize
# сравним с nltk
from nltk import word_tokenize, sent_tokenize

In [ ]:
text = 'Разве это правильно? Написано 0,5л, а вы приносите 0,45л, за те же деньги'
list(tokenize(text))

In [ ]:
word_tokenize(text)

Одна из полезных штук здесь - это получение индексов начала и конца токена. Для чего это может быть нужно?

In [ ]:
text = '''
Кто вообще в 21 веке пользуется pymorphy или natasha, если есть гпт?
Я с ним все домашки делаю и нормально.
'''
list(sentenize(text))

In [ ]:
sent_tokenize(text)

__Задание 3:__ Как думаете, какие тексты обычно вызывают затруднения при делении на предложения? Попробуйте привести пример такого небольшого текста, чтобы razdel и nltk по-разному его разделили



In [ ]:
text = # your text here
list(sentenize(text))

In [ ]:
sent_tokenize(text)

!НО! razdel хорошо работает только с качественно написанными текстами:(

In [ ]:
text = 'Мама купила платье. оно красивое.'
list(sentenize(text))

In [ ]:
sent_tokenize(text)

## Slovnet

In [ ]:
!wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

[__Slovnet__](https://habr.com/ru/articles/516098/#slovnet) - библиотека с моделями для решения NER задач, а также морфологического и синтаксического парсинга. Отличия от подходов с использованием нейросетей типа Bert - скорость и маленький вес модели. В качестве входных данных использует предобученные эмбеддинги navec.

Проблема: опять же работает на качественных данных, типа художественной литературы и новостей, так как не решает проблему OOV. Если слово не находится в словаре, то заменяется на "\<unk\>"

In [ ]:
import navec

from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup

# path_navec = 'navec_hudlit_v1_12B_500K_300d_100q.tar

path_navec = 'navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path_navec)
path_slovnet = 'slovnet_ner_news_v1.tar'
ner = NER.load(path_slovnet)
ner.navec(navec)

In [ ]:
text = 'У моего любимого певца Егора Крида вышел новый трек. Буду его слушать весь день!'

markup = ner(text)
show_markup(markup.text, markup.spans)

## Yargy - Поиск имён, дат и адресов

Если много однотипных данных, которые можно описать правилами, то использовать сложный NER, тратить время на разметку и обучение необязательно. Yargy - система описательных правил, что-то типа умных регулярок, только быстрее и удобнее. Помимо этого, они ещё приводят всё к одному виду - его в целом тоже можно придумать самим.

Есть некоторые уже готовые правила:

In [ ]:
from natasha import (
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor
)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

### Даты

In [ ]:
text = '24.01.2017, 2015 год, 2014 г, 1 апреля, май 2017 г., 9 мая 2017 года'
matches = list(dates_extractor(text))

In [ ]:
matches[:2]

In [ ]:
for date in matches:
    day, month, year = date.fact.day, date.fact.month, date.fact.year
    print(f'{day}.{month}.{year}')

### Имена

In [ ]:
text = 'Меня зовут Ксюша Шерман. Или Шерман Ксюша. Или даже Ксения Валерьевна Шерман'
list(names_extractor(text))

Но почему-то его иногда клинит

In [ ]:
text = 'а может и не Ксюша'
list(names_extractor(text))

### Адреса

In [ ]:
text = 'Я живу в г.Москве, на ул. Академическая'
list(addr_extractor(text))

__Задание 4:__ Снова ваша очередь придумать такой адрес, который ему нормально не удастся распарсить.

In [ ]:
text = # your text here
list(addr_extractor(text))

### Денежные выражения

In [ ]:
text = 'у меня есть 250р.'
list(money_extractor(text))

In [ ]:
text = 'у меня есть 250 тысяч рублей'
list(money_extractor(text))

In [ ]:
text = 'у меня есть 250 тенге'
list(money_extractor(text))

Можно так же придумать свои правила. Есть достаточно понятные описания от создателей библиотеки.
Стоит помнить, что готовые правила работают, опять же, с качественными данными типа новостей или юридических документов (что, кстати, является одной из популярных задач).

## Spacy

In [ ]:
!pip install spacy -q
!python -m spacy download ru_core_news_sm -q

In [ ]:
import spacy

# Загружаем весь пайплайн для русского
nlp = spacy.load("ru_core_news_sm")

# Обрабатываем текст
text = 'У моего любимого певца Егора Крида вышел новый трек. Буду его слушать весь день!'
doc_spacy = nlp(text)

# Выведем токены, леммы и теги
for i, s in enumerate(doc_spacy.sents):
    print("\n-- Sentence %d --" % i)
    for t in s:
        print(t.text, t.pos_, t.dep_)

In [ ]:
text = 'У моего любимого певца Егора Крида вышел новый трек. Буду его слушать весь день!'
doc_natasha = Doc(text)

doc_natasha.segment(segmenter)
doc_natasha.tag_morph(morph_tagger)
doc_natasha.parse_syntax(syntax_parser)

for i, s in enumerate(doc_natasha.sents):
    print("\n-- Sentence %d --" % i)
    for t in s.tokens:
        print(t.text, t.pos, t.rel, sep="\t")

__Вопрос:__ Проанализируйте результаты. Почему они могут быть такие?

### Лемматизация

In [ ]:
for i, s in enumerate(doc_spacy.sents):
    print("\n-- Sentence %d --" % i)
    for t in s:
        print(t.text, t.lemma_, t.lemma, sep="\t")

### Морфология

In [ ]:
for i, s in enumerate(doc_spacy.sents):
    print("\n-- Sentence %d --" % i)
    for t in s:
        print(t.text, t.morph)

### Синтаксис

In [ ]:
for token in doc_spacy:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

__Вопрос:__ А что такое "xxxx" и подобное в выводе?

In [ ]:
from spacy import displacy

displacy.serve(doc_spacy, style="dep")

In [ ]:
displacy.serve(doc_spacy, style="ent")

In [ ]:
for token in doc_spacy:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

__Задание 5:__ Выберите любую понравившуюся библиотеку - natasha или spacy и попробуйте достать все пары "прилагательное+существительное" из предложения:

In [ ]:
text = 'У моего любимого певца Егора Крида вышел новый трек. Буду его слушать весь день сегодняшний!'

# YOUR CODE HERE #